In [8]:
import config
import pymongo
import numpy as np
import pandas as pd
import json
import re
import simplejson
from math import isnan
import ast
#import attributes
from models import Artefact, Vondst, Spoor

import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection
from sqlalchemy.types import VARCHAR

from pandas_profiling import ProfileReport
from ipywidgets import interact, Dropdown

from sqlalchemy import and_, or_


In [23]:
db = create_engine(config.SQLALCHEMY_DATABASE_URI)
rs = None

Session = sessionmaker(bind=db)
session = Session()

rs = (session.query(Artefact.primary_key, Spoor.spoordatering_vanaf, Spoor.spoordatering_tot)
      .join(Vondst, Vondst.primary_key==Artefact.vondstID)
      .join(Spoor, Spoor.primary_key==Vondst.spoorID)
      .filter(or_(Spoor.spoordatering_vanaf != None, Spoor.spoordatering_tot != None))
)


#session.query(Artefact.primary_key, Spoor.spoordatering_vanaf, Spoor.spoordatering_tot).select_from(select_from.(Vondst).join(Spoor)))        
for row in rs:
    print(row)
    

In [18]:
rs

In [22]:
rs = (session.query(Artefact.primary_key, Vondst.vondstdatering_vanaf, Vondst.vondstdatering_tot)
      .join(Vondst, Vondst.primary_key==Artefact.vondstID)
      .filter(or_(Vondst.vondstdatering_vanaf != None, Vondst.vondstdatering_tot != None))
)


#session.query(Artefact.primary_key, Spoor.spoordatering_vanaf, Spoor.spoordatering_tot).select_from(select_from.(Vondst).join(Spoor)))        
for row in rs:
    print(row)
    

(9081, 1600, 1650)
(12092, 1600, 1650)
(12397, 1600, 1650)
(10610, 1600, 1650)
(11719, 1600, 1650)


In [25]:
query = (Artefact.query(Artefact.primary_key, Vondst.vondstdatering_vanaf, Vondst.vondstdatering_tot)
      .join(Vondst, Vondst.primary_key==Artefact.vondstID)
      .filter(or_(Vondst.vondstdatering_vanaf != None, Vondst.vondstdatering_tot != None))
)
        
query.update({"acct_active": True}, synchronize_session=False)

AttributeError: type object 'Artefact' has no attribute 'query'